# 🏠 Mortgage Risk Modeling (MRM)

This project builds a mortgage risk workflow:

- Read and combine loan-level performance data (5 states, 2020Q1)
- Define a loan-level **bad** target from delinquency history
- Explore distributions and data quality
- Prepare modeling datasets
- Build segmentation (KMeans) and deep-learning models (ANN/CNN)

> Notebooks are organized as `mrm_01` … `mrm_06` for a clean, portfolio-ready structure.

## Project structure

- `mrm_01_project_overview.ipynb` — goals, data sources, definitions
- `mrm_02_exploratory_data_analysis.ipynb` — EDA + data quality checks
- `mrm_03_data_wrangling.ipynb` — cleaning, dedupe, target creation
- `mrm_04_feature_engineering.ipynb` — transformations + scaling
- `mrm_05_modeling.ipynb` — KMeans segmentation validation
- `mrm_06_deep_learning.ipynb` — ANN/CNN + score combination

## Setup

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
\
# ⚠️ Update your local data path
# This repo assumes the Freddie Mac Single-Family Loan-Level Dataset CSV extracts
# are saved locally. Update `DATA_DIR` to your machine.

DATA_DIR = r"C:\\Users\\aleci\\Documents\\Project 3 - Mortgage Origination Analysis\\data\\"

In [ ]:
# Read 2020Q1 state files and combine
file_2020q1_OH = pd.read_csv(DATA_DIR + "file_2020Q1_OH.csv")
file_2020q1_VA = pd.read_csv(DATA_DIR + "file_2020Q1_VA.csv")
file_2020q1_GA = pd.read_csv(DATA_DIR + "file_2020Q1_GA.csv")
file_2020q1_MI = pd.read_csv(DATA_DIR + "file_2020Q1_MI.csv")
file_2020q1_CO = pd.read_csv(DATA_DIR + "file_2020Q1_CO.csv")

df_all = pd.concat([file_2020q1_OH, file_2020q1_VA, file_2020q1_GA, file_2020q1_MI, file_2020q1_CO], ignore_index=True)
df_all.shape

In [ ]:
df_all['LOAN_ID'].nunique()

## Target definition (loan-level)

We define a loan as **bad** if it ever reaches `DLQ_STATUS >= 6` across the observed monthly records.

In [ ]:
# Define "bad" at the monthly observation level:
# A loan-month is "bad" if delinquency status >= 6 (i.e., 180+ DPD equivalent)
df_all["bad"] = np.where(df_all["DLQ_STATUS"] >= 6, 1, 0)

# Collapse to loan-level target: if any month is bad -> loan is bad
df_bad = df_all.groupby("LOAN_ID")["bad"].sum().reset_index()
df_bad["bad"] = np.where(df_bad["bad"] > 0, 1, 0)

df_bad["bad"].mean(), df_bad["bad"].sum()